In [7]:
import socket
import tqdm
from Crypto.Cipher import AES
import pickle
import struct
import torch
from torch import nn

In [2]:
# setting key and nonce of 16 bytes
key = b"FedLwithHEandIoT"
nonce = b"FedLwithHEandIoT"

In [3]:
host = socket.gethostbyname(socket.gethostname())
port = 10080
print(host)

192.168.0.120


In [4]:
server_soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_soc.bind((host, port))
server_soc.listen()
print("Listening")
client, addr = server_soc.accept()
print('Conntected with', addr)

Listening
Conntected with ('192.168.0.120', 64147)


In [5]:
def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4, decrypt=False)
    if not raw_msglen:
        return None
    
    msglen = struct.unpack('>I', raw_msglen)[0]

    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n, decrypt=True):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
        
    if decrypt == True:
        engine = AES.new(key, AES.MODE_EAX, nonce)
        data = engine.decrypt(data)
  
    return data

In [6]:
weights = recv_msg(client)
print(f"weights: {weights}")

client.close()
server_soc.close()

weights: OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])


In [8]:
# First pyTorch model (Linear regression)
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad=True,
                                             dtype=torch.float))
    # To define computation in the model
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

In [9]:
loaded_model = LinearRegressionModel()

In [10]:
loaded_model.state_dict()

OrderedDict([('weights', tensor([-0.1352])), ('bias', tensor([-0.9152]))])

In [11]:
MODEL_PATH = "01_pytorch_workflow_model_0.pth"
loaded_model.load_state_dict(torch.load(f=MODEL_PATH))

<All keys matched successfully>

In [12]:
loaded_model.state_dict()

OrderedDict([('weights', tensor([0.6990])), ('bias', tensor([0.3093]))])